In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import pandas as pd

In [59]:
# Load the data from eclipse.csv
eclipse_data = pd.read_csv("eclipse.csv")

# Load the pairs data from eclipse_pairs.csv
pairs_data = pd.read_csv("eclipse_pairs.csv")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4728\2971479914.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,224,225,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,243,244,246,247,248,249,250,251,252,253,254,255,257,258,

In [60]:
# print the description of the row that has id 214611
print(eclipse_data[eclipse_data["bug_id"] == 214611]["description"].values[0])


Build ID: Build id: M20071023-1652

Steps To Reproduce:
1. navigating between eclipse-views. i dont't know which action exactly brings up this error


More information:
!ENTRY org.eclipse.ui.workbench 4 2 2008-01-08 15:22:23.471
!MESSAGE Problems occurred when invoking code from plug-in: "org.eclipse.ui.workbench".
!STACK 0
java.lang.NoSuchMethodError: org.eclipse.mylyn.internal.context.core.InteractionContextManager.getScalingFactors()Lorg/eclipse/mylyn/internal/context/core/ScalingFactors;
	at org.eclipse.mylyn.internal.team.ui.ContextActiveChangeSetManager.shouldRemove(ContextActiveChangeSetManager.java:261)
	at org.eclipse.mylyn.internal.team.ui.ContextActiveChangeSetManager.interestChanged(ContextActiveChangeSetManager.java:229)
	at org.eclipse.mylyn.internal.context.core.InteractionContextManager.notifyInterestDelta(InteractionContextManager.java:391)
	at org.eclipse.mylyn.internal.context.core.InteractionContextManager.processInteractionEvent(InteractionContextManager.java:312)


In [61]:
# Function to preprocess bug description text
def preprocess_bug_description(description):

    # Convert to lowercase
    description = description.lower()

    # Remove punctuation and special characters
    description = re.sub(r'[^a-zA-Z0-9\s]', '', description)

    # Remove numbers
    description = re.sub(r'\b\d+\b', '', description)

    # Remove extra whitespace
    description = re.sub(r'\s+', ' ', description)

    return description

In [62]:
# Preprocess Bug Reports
def preprocess_bug_reports(bug_reports):
    preprocessed_reports = []
    for report in bug_reports:
        # Implement your preprocessing steps here
        preprocessed_report = preprocess_bug_description(report)
        preprocessed_reports.append(preprocessed_report)
    return preprocessed_reports

In [63]:
# Create a Vocabulary and Generate BoW Vectors
def generate_bow_vectors(bug_reports):
    # Initialize CountVectorizer to create BoW vectors
    vectorizer = CountVectorizer()
    # Fit the vectorizer on preprocessed bug reports to build the vocabulary
    vectorizer.fit(bug_reports)
    # Transform bug reports into BoW vectors
    bow_vectors = vectorizer.transform(bug_reports)
    return bow_vectors

In [64]:
# Calculate Similarity
def calculate_similarity(bow_vectors):
    # Calculate cosine similarity between each pair of bug reports
    num_reports = bow_vectors.shape[0]
    similarities = []
    for i in range(num_reports):
        for j in range(i+1, num_reports):
            similarity_score = cosine_similarity(bow_vectors[i], bow_vectors[j])[0][0]
            similarities.append(similarity_score)
    return similarities

In [65]:
# Function to get bug descriptions based on bug IDs
def get_bug_description(bug_id):

    # if bug_id is NaN, return None
    if pd.isnull(bug_id):
        return None

    # cast bug_id to int
    bug_id = int(bug_id)

    # if id is not found in the dataset, return None
    if bug_id not in eclipse_data['bug_id'].values:
        return None
    
    return eclipse_data[eclipse_data['bug_id'] == bug_id]['description'].values[0]


In [66]:
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Iterate through pairs data and perform preprocessing, BOW, and cosine similarity
for index, row in pairs_data.iterrows():
    issue_id = row['issue_id']
    duplicate_id = row['duplicate']

    if pd.isnull(duplicate_id):
        continue

    duplicate_ids = row['duplicate'].split(';')

    # Iterate through each duplicate ID
    for duplicate_id in duplicate_ids:
        duplicate_id = duplicate_id.strip()  # Remove any spaces left or right    

        print(f"Comparing Bug {issue_id} and Bug {duplicate_id}")
        
        # Get descriptions of the bugs
        description1 = get_bug_description(issue_id)
        description2 = get_bug_description(duplicate_id)

        # if one of the descriptions is None, skip the comparison    
        if description1 is None or description2 is None:
            print("One of the descriptions is not found in the dataset. Skipping the comparison.")
            continue
    
        # Preprocess bug descriptions
        preprocessed_desc1 = preprocess_bug_description(description1)
        preprocessed_desc2 = preprocess_bug_description(description2)

        # Check if the documents contain only stop words
        if all(word in vectorizer.get_stop_words() for word in preprocessed_desc1.split()) or \
            all(word in vectorizer.get_stop_words() for word in preprocessed_desc2.split()):
            print("Documents contain only stop words. Skipping the comparison.")
            continue
        # Create BOW representation for each bug report
        bow_corpus = [preprocessed_desc1, preprocessed_desc2]
        bow_matrix = vectorizer.fit_transform(bow_corpus)
        
        # Get BOW vectors for each bug report
        bow1 = bow_matrix[0].toarray()
        bow2 = bow_matrix[1].toarray()
        
        # Compute cosine similarity between BOW representations
        similarity = cosine_similarity(bow1, bow2)[0][0]
        
        print(f"Similarity between Bug {issue_id} and Bug {duplicate_id}: {similarity}")


Comparing Bug 214301 and Bug 214611
Similarity between Bug 214301 and Bug 214611: 0.9772016464917131
Comparing Bug 214445 and Bug 214451
Similarity between Bug 214445 and Bug 214451: 0.5326219069229958
Comparing Bug 214466 and Bug 214452
Similarity between Bug 214466 and Bug 214452: 0.2928195306397773
Comparing Bug 214577 and Bug 217601
One of the descriptions is not found in the dataset. Skipping the comparison.
Comparing Bug 214862 and Bug 214759
Similarity between Bug 214862 and Bug 214759: 0.37818779029489086
Comparing Bug 215052 and Bug 214414
Similarity between Bug 215052 and Bug 214414: 0.45556232589300527
Comparing Bug 215052 and Bug 213299
One of the descriptions is not found in the dataset. Skipping the comparison.
Comparing Bug 214411 and Bug 216725
One of the descriptions is not found in the dataset. Skipping the comparison.
Comparing Bug 214411 and Bug 216663
One of the descriptions is not found in the dataset. Skipping the comparison.
Comparing Bug 214411 and Bug 210304
O

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [19]:
# Sample bug report data
bug_reports = [
    """Bug report 1: UnknownHostException is not caught when the host name is invalid

1. in the Repositories view, New->CVS repository location
2. put a garbage host name( e.g. eh? )
3. complete the wizard.
4. expand the ""Project Version""
	UHE is thrown.

Log: Wed Jun 06 12:51:50 EDT 2001
4 org.eclipse.vcm.core 4 Cannot locate host eh?
java.net.UnknownHostException: eh?
	at java.net.InetAddress.getAllByName0(InetAddress.java:585)
	at java.net.InetAddress.getAllByName0(InetAddress.java:554)
	at java.net.InetAddress.getByName(InetAddress.java:463)
	at java.net.Socket.<init>(Socket.java:112)
	at 
org.eclipse.vcm.internal.core.ccvs.client.PServerConnection.createSocket
(PServerConnection.java:140)
	at org.eclipse.vcm.internal.core.ccvs.client.PServerConnection.open
(PServerConnection.java:190)
	at org.eclipse.vcm.internal.core.ccvs.client.Connection.open
(Connection.java:224)
	at org.eclipse.vcm.internal.core.ccvs.client.Client.<init>
(Client.java:226)
	at 
org.eclipse.vcm.internal.core.ccvs.CVSAdapter.internalFetchMemberNames
(CVSAdapter.java:768)
	at 
org.eclipse.vcm.internal.core.ccvs.CVSAdapter.fetchTeamStreamProjectNames
(CVSAdapter.java:591)
	at org.eclipse.vcm.internal.core.TeamStream.fetchProjects
(TeamStream.java:58)
	at 
org.eclipse.vcm.internal.ui.model.TeamStreamElement.internalGetChildren
(TeamStreamElement.java:82)
	at org.eclipse.vcm.internal.ui.model.TeamStreamElement$1.run
(TeamStreamElement.java:39)
	at org.eclipse.swt.custom.BusyIndicator.showWhile(BusyIndicator.java:97)
	at org.eclipse.vcm.internal.ui.model.TeamStreamElement.getChildren
(TeamStreamElement.java:37)
	at org.eclipse.ui.model.WorkbenchContentProvider.getChildren
(WorkbenchContentProvider.java:53)
	at org.eclipse.jface.viewers.AbstractTreeViewer.getRawChildren
(AbstractTreeViewer.java:568)
	at org.eclipse.jface.viewers.StructuredViewer.getFilteredChildren
(StructuredViewer.java:281)
	at org.eclipse.jface.viewers.StructuredViewer.getSortedChildren
(StructuredViewer.java:378)
	at org.eclipse.jface.viewers.AbstractTreeViewer.createChildren
(AbstractTreeViewer.java:238)
	at org.eclipse.jface.viewers.AbstractTreeViewer.handleTreeExpand
(AbstractTreeViewer.java:611)
	at org.eclipse.jface.viewers.AbstractTreeViewer$2.treeExpanded
(AbstractTreeViewer.java:631)
	at org.eclipse.swt.widgets.TypedListener.handleEvent(TypedListener.java
(Compiled Code))
	at org.eclipse.swt.widgets.Widget.sendEvent(Widget.java(Compiled Code))
	at org.eclipse.swt.widgets.Widget.sendEvent(Widget.java(Compiled Code))
	at org.eclipse.swt.widgets.Tree.wmNotifyChild(Tree.java(Compiled Code))
	at org.eclipse.swt.widgets.Composite.WM_NOTIFY(Composite.java(Compiled 
Code))
	at org.eclipse.swt.widgets.Composite.WM_NOTIFY(Composite.java(Compiled 
Code))
	at org.eclipse.swt.widgets.Control.windowProc(Control.java(Compiled 
Code))
	at org.eclipse.swt.widgets.Display.windowProc(Display.java(Compiled 
Code))
	at org.eclipse.swt.internal.win32.OS.CallWindowProc(Native Method)
	at org.eclipse.swt.widgets.Tree.callWindowProc(Tree.java(Compiled Code))
	at org.eclipse.swt.widgets.Tree.WM_LBUTTONDOWN(Tree.java:1043)
	at org.eclipse.swt.widgets.Control.windowProc(Control.java(Compiled 
Code))
	at org.eclipse.swt.widgets.Display.windowProc(Display.java(Compiled 
Code))
	at org.eclipse.swt.internal.win32.OS.DispatchMessage(Native Method)
	at org.eclipse.swt.widgets.Display.readAndDispatch(Display.java
(Compiled Code))
	at org.eclipse.ui.internal.Workbench.runEventLoop(Workbench.java
(Compiled Code))
	at org.eclipse.ui.internal.Workbench.run(Workbench.java:620)
	at org.eclipse.core.internal.boot.InternalBootLoader.run
(InternalBootLoader.java:815)
	at org.eclipse.core.boot.BootLoader.run(BootLoader.java:285)
	at java.lang.reflect.Method.invoke(Native Method)
	at org.eclipse.core.launcher.Main.basicRun(Main.java:69)
	at org.eclipse.core.launcher.Main.run(Main.java:311)
	at org.eclipse.core.launcher.Main.main(Main.java:198)

NOTES:

KM (6/7/01 2:50:06 PM)
	Recommend we catch at level of TeamStreamElement.internalGetChildren() 
and do
			ErrorDialog.openError(...)
	Recommend that class because its only UI place we can catch, and its 
already catching CoreExceptions there anyway.

	There are other places you could get this error since we lazy verify 
connections but this is the most common.

	Also, noticed that in VCMModelElement.handle() (where this IS being 
caught)
	we should not be loging info messages, only errors.

KH (6/8/2001 5:47:05 PM)
	Yes """,
    
    """Bug report 2: After having synchronized and released successfully with teamstream (on zrhcvs), I attempted to version the project org.eclipse.jdt.core
and got a dialog saying ""CVS communication error"", following stack trace was in the log:

Log: Sat Jun 02 00:45:55 CEST 2001
4 org.eclipse.vcm.core 4 CVS communication error
java.io.InterruptedIOException: Read timed out
	at java.net.SocketInputStream.socketRead(Native Method)
	at java.net.SocketInputStream.read(SocketInputStream.java(Compiled Code))
	at java.io.BufferedInputStream.fill(BufferedInputStream.java(Compiled Code))
	at java.io.BufferedInputStream.read(BufferedInputStream.java(Compiled Code))
	at org.eclipse.vcm.internal.core.ccvs.client.Connection.readLineOrUntil(Connection.java(Compiled Code))
	at org.eclipse.vcm.internal.core.ccvs.client.Connection.readToken(Connection.java(Compiled Code))
	at org.eclipse.vcm.internal.core.ccvs.client.Client.processResponses(Client.java(Compiled Code))
	at org.eclipse.vcm.internal.core.ccvs.client.Client.processResponses(Client.java:397)
	at org.eclipse.vcm.internal.core.ccvs.client.Client.execute(Client.java:257)
	at org.eclipse.vcm.internal.core.ccvs.CVSAdapter.createProjectVersion(CVSAdapter.java:313)
	at org.eclipse.vcm.internal.core.SharingManager.createProjectVersion(SharingManager.java:71)
	at org.eclipse.vcm.internal.ui.actions.VersionAction$VersionLocalOperation.version(VersionAction.java:121)
	at org.eclipse.vcm.internal.ui.VersionResourcesOperation.run(VersionResourcesOperation.java:45)
	at org.eclipse.jface.operation.ModalContext$ModalContextThread.run(ModalContext.java:98)

NOTES:

PM (6/2/2001 12:48:47 AM)
	My network connection did not drop. I am connected over VPN from home to OTT.
	Next versioning attempt worked fine""",
    
    """Bug report 3: When the connection to the repositories fails, InterruptedIOException is not caught.

Log: Wed Jun 13 12:13:40 EDT 2001
4 org.eclipse.vcm.core 4 Cannot connect to host cvslnxtest
java.io.InterruptedIOException: Operation timed out: no further information
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.PlainSocketImpl.doConnect(PlainSocketImpl.java:329)
	at java.net.PlainSocketImpl.connectToAddress(PlainSocketImpl.java:141)
	at java.net.PlainSocketImpl.connect(PlainSocketImpl.java:128)
	at java.net.Socket.<init>(Socket.java:285)
	at java.net.Socket.<init>(Socket.java:112)
	at org.eclipse.vcm.internal.core.ccvs.client.PServerConnection.createSocket(PServerConnection.java:140)
	at org.eclipse.vcm.internal.core.ccvs.client.PServerConnection.open(PServerConnection.java:190)
	at org.eclipse.vcm.internal.core.ccvs.client.Connection.open(Connection.java:224)
	at org.eclipse.vcm.internal.core.ccvs.client.Client.<init>(Client.java:227)
	at org.eclipse.vcm.internal.core.ccvs.CVSAdapter.internalFetchMemberNames(CVSAdapter.java:778)
	at org.eclipse.vcm.internal.core.ccvs.CVSAdapter.fetchProjectNames(CVSAdapter.java:485)
	at org.eclipse.vcm.internal.core.Repository.fetchProjectList(Repository.java:128)
	at org.eclipse.vcm.internal.ui.model.ProjectVersionsCategory.internalGetChildren(ProjectVersionsCategory.java:92)
	at org.eclipse.vcm.internal.ui.model.ProjectVersionsCategory$1.run(ProjectVersionsCategory.java:48)
	at org.eclipse.swt.custom.BusyIndicator.showWhile(BusyIndicator.java(Compiled Code))
	at org.eclipse.vcm.internal.ui.model.ProjectVersionsCategory.getChildren(ProjectVersionsCategory.java:46)
	at org.eclipse.ui.model.WorkbenchContentProvider.getChildren(WorkbenchContentProvider.java:51)
	at org.eclipse.jface.viewers.AbstractTreeViewer.getRawChildren(AbstractTreeViewer.java:568)
	at org.eclipse.jface.viewers.StructuredViewer.getFilteredChildren(StructuredViewer.java(Compiled Code))
	at org.eclipse.jface.viewers.StructuredViewer.getSortedChildren(StructuredViewer.java(Compiled Code))
	at org.eclipse.jface.viewers.AbstractTreeViewer.createChildren(AbstractTreeViewer.java(Compiled Code))
	at org.eclipse.jface.viewers.AbstractTreeViewer.handleTreeExpand(AbstractTreeViewer.java:611)
	at org.eclipse.jface.viewers.AbstractTreeViewer$2.treeExpanded(AbstractTreeViewer.java:631)
	at org.eclipse.swt.widgets.TypedListener.handleEvent(TypedListener.java(Compiled Code))
	at org.eclipse.swt.widgets.Widget.sendEvent(Widget.java(Compiled Code))
	at org.eclipse.swt.widgets.Widget.sendEvent(Widget.java(Compiled Code))
	at org.eclipse.swt.widgets.Tree.wmNotifyChild(Tree.java(Compiled Code))
	at org.eclipse.swt.widgets.Composite.WM_NOTIFY(Composite.java(Compiled Code))
	at org.eclipse.swt.widgets.Composite.WM_NOTIFY(Composite.java(Compiled Code))
	at org.eclipse.swt.widgets.Control.windowProc(Control.java(Compiled Code))
	at org.eclipse.swt.widgets.Display.windowProc(Display.java(Compiled Code))
	at org.eclipse.swt.internal.win32.OS.CallWindowProc(Native Method)
	at org.eclipse.swt.widgets.Tree.WM_LBUTTONDOWN(Tree.java(Compiled Code))
	at org.eclipse.swt.widgets.Tree.WM_LBUTTONDOWN(Tree.java(Compiled Code))
	at org.eclipse.swt.widgets.Control.windowProc(Control.java(Compiled Code))
	at org.eclipse.swt.widgets.Display.windowProc(Display.java(Compiled Code))
	at org.eclipse.swt.internal.win32.OS.DispatchMessage(Native Method)
	at org.eclipse.swt.widgets.Display.readAndDispatch(Display.java(Compiled Code))
	at org.eclipse.ui.internal.Workbench.runEventLoop(Workbench.java(Compiled Code))
	at org.eclipse.ui.internal.Workbench.run(Workbench.java:620)
	at org.eclipse.core.internal.boot.InternalBootLoader.run(InternalBootLoader.java:815)
	at org.eclipse.core.boot.BootLoader.run(BootLoader.java:285)
	at java.lang.reflect.Method.invoke(Native Method)
	at org.eclipse.core.launcher.Main.basicRun(Main.java:126)
	at org.eclipse.core.launcher.Main.run(Main.java:408)
	at org.eclipse.core.launcher.Main.main(Main.java:281)""",
     """Bug report 4: "Since refresh() is called before the catchup/release view opens, it should be as fast as possible.

NOTES:

BB (12/04/2001 11:32:05 AM)
	The most promising optimization idea is: Don't return an IServerResourceTree from fetchRemoteTree
	but instead use a call-back to interleave the server communication with updating the sync info.

BB (12/04/2001 11:30:31 AM)
	See this PR for other performance tuning opportunities: 1GAT2V2: ITPVCM:WINNT - Releasing very slow

MichaelV (19/09/2001 2:14:23 PM)

	Here are some percentages on a refresh from OTT to Paris with auto-build turned OFF

		Total Time: 97776 ms

		(57026) 58.3% in Subscriber.refresh()
			(40704) 41.6% CVSAdapter.fetchRemoteTree()
			(14918) 15.3% Subscriber.recursiveRefresh()
		(13242) 13.5% in SharingManager.manage() from  org.eclipse.vcm.ui.sync.Repository.autoManage()
			What is autoManage?
		 (12234) 12.5% in org.eclipse.vcm.ui.sync.Repository.collectResourceChanges() (recursive)
		11% in JavaModelManager.resourceChanged()
		4.3% in ResourceDeltaFactory.computeDelta()

	Hot-spots are
		14% socket read
		13% socket write
		12% String#substring
		4.6% String#indexOf
		3.6% Path#segments
		3.2% String#compareTo
		3.2% AccumulatingProgressMonitor#subtask
		3% Path#append
		3.2% String#concat

	Auto manage time (13.5%) seems to be spent determining what is ignored. There seems to be many ways to improve this!
	
MichaelV (20/09/2001 8:51:55 AM)

	There seems to be a lot of room for improvement in Subscriber#recursiveRefresh
	However, the general approach used by VCM to determine what changes there are seems to be
	inherently ineficient.

	In CVS clients, the following steps take place on an update
	1. Client sends information about all loaded files (versions and what has changed locally)
	2. Server makes one pass through to determine update reponse.
	3. Client uses response to update only as necessary
	This results in 2 to 3 passes through the project

	In VCM, the following occurs
	1. client sends information about known directories
	2. Server responds with information about new directories and files
	3. client requests file information (status)
	4. server responds + client must place file in resulting tree
	5. resulting tree + local structure traversed to update sync info
	6. local structure traversed to determine if their are changes
	My guess is that this is roughly equivalent to 5 passes

	Considering that some projects may be very large, this leads to slow syncing 
	which takes the same time whether there are changes or not.
	
MichaelV (21/09/2001 2:06:02 PM)

	I have optimized the recursive refresh itself but this has bought little (13000). The problem is
	that we do a get and write of the sync info at each node and this is costly. Furthermore,
	getting and setting get slower as the distance from the root increases. This is probably due
	to the fact that there are lots of files at the leaves. 

	Within getWorkspaces, getSharing and getProject are the main culprits. Can these be passed in?
	The project should be OK but I'm unsure of the sharing. getRepository also does a getProject.

	I have optimized SharingManager.manage to avoid internalGetSyncInfo and getProject. (from 13000 to 9000)
	However, we should not need a separate traversal to ensure that all non-ignored resources are managed!

	I have included change collection as part of the refresh. 
	(Much speed improvement!! from 12000 to virtualy 0)

MichaelV (24/09/2001 12:49:57 PM)
	
	Timings of local sync with auto-manage refresh and collection in same pass.

	100% = 30290 ms
	82.73% (25059) in Subscriber.refresh
		49.95% (15130) in CVSAdapter.fetchRemoteTree
		27.25% (8257) in Subscriber.refreshChildren
		5% (1512) in Policy.countResources
	15.34% (4649) in NotificationManager.getDelta()

	This is a savings of over 75% on VCM operations but no improvement in fetchRemoteTree. 
	Also, getDelta timings result from all the storeSyncInfo calls. One solution is
	to request only the delta information from the server as the CVS command-line
	client does.
 
Test Cases

	Sync for first time
	- on unshared project
	- on folder/file of unshared project
	- on folder/file of shared project

	Sync with outgoing, incomming and both

	Merges with outgoing, incomming and both"
""",
]

# ---------------------------------------------
bug_reports2 = [
     """Bug Report 1: Thought it would be useful if the set of repo connections could be stored somewhere
external in the file system instead of everyone having their own list cashed in
their workspace (.metadata).  Thus if you get a new workspace you don't need to reconstruct
this information, you can just configure your workspace to point to that list.
Also, it means a team such as ours could share a set of repo connections by
all pointing to the same file.

NOTES:
""",
"""Bug Report 2: I would like to be able to connect to a team stream and directly add its content to the
	workspace. This could be added to the ""New repository wizard"".
	Currently I need at least 5 steps to populate a new workspace with a team stream:
	1. Add repository
	2. Open repository browser
	3. Expand HEAD (see PR: 1GCQK5Y: ITPVCM:WINNT - Should be able to add HEAD to workspace)
	4. Select all children of HEAD
	5. Add to workspace


NOTES:

KM (4/25/01 2:02:28 PM)
	Moving to future.

BB (4/25/01 12:27:47 PM)
	At least for CVS, a team stream cannot be used as a container of projects, so this operation would not
	be meaningful for a CVS repository. Add to workspace is not a frequent operation (unless you're self-hosting),
	I don't think 5 steps is too bad for this.
""",
     
]

bug_reports3 = [
    """Bug report 1: Project descriptions don't store sharing recommendations and project version references yet, but they should.

NOTES:

Jean-Michel (4/6/01 12:39:12 PM)
	We will live with unqualified project references for now.""",
    """Bug report 2: "Project references come in three flavours:

1. project name
2. project name + version (+ repo)
3.  project name + stream (+ repo)

We currently store #1.  While noone makes use of form 2 or 3, we should ensure that being able to store them at some
future time will not require changing our .vcm_meta file format.

This could be solved, for example, by having distinct xml tags for each of these forms.

NOTES:

BB (5/14/2001 4:13:08 PM)
	Right now, project references of type 1 are stored as: <project-reference name=""...""/>
	If we use different tag names for 2. and 3. we should be fine. For example:
<project-version-reference name=""projectname"" versionname=""v1"" repository="":pserver:user@cvslnx:/home/cvs/vcmrepo""/>
	Moving to Active-TimePermitting as a reminder to add project references of type 2 and 3."
""",
]

# Preprocess bug reports
preprocessed_bug_reports = preprocess_bug_reports(bug_reports3)

# Generate BoW vectors
bow_vectors = generate_bow_vectors(preprocessed_bug_reports)

# Calculate similarity
similarities = calculate_similarity(bow_vectors)

# Print similarity scores between each pair of bug reports and the index of the bug reports
num_reports = bow_vectors.shape[0]
for i in range(num_reports):
	for j in range(i+1, num_reports):
		print(f"Similarity between bug report {i+1} and {j+1}: {similarities.pop(0)}")
# ---------------------------------------------


Similarity between bug report 1 and 2: 0.5091331749765979
